# Slides [here](https://docs.google.com/presentation/d/1iLMu2MvnkyjGzEf__e1ftTEVtV20TB3QAfSqJm24l54/edit?usp=sharing)

In [ ]:
import pandas as pd
import numpy as np
netflix  = pd.read_csv("Netflix-Shows-2.csv", encoding= "ISO-8859-1")
movies = pd.read_csv("movies.csv",  encoding= "ISO-8859-1")

In [ ]:
netflix.head()

In [ ]:
movies.head()

# Exploratory Data Analysis: Column Functions

Let's start by investing what our column types are. We can do this using the **df.dtype** function.

In [ ]:
netflix.dtypes

In [ ]:
movies.dtypes

Notice that columns such as the movie name and company are object types, meaning that it is an immutable type such as string. 

## Converting Data Types
There are two standard ways of converting pandas data types:
<br>
1) \<column>.astype(\<desired type>)
<br>
2) conversion helper functions, like pd.to_numeric or pd.to_datetime



Let's start with the .astype() function, which has to be called directly on the column that you want to convert. We must then re-assign the column to the converted column. 
<br>
<br>
 Let's convert the 'ratingDescription' column of the netflix dataset from an integer to a float. 

In [ ]:
netflix['ratingDescription'].astype('float')

Exercise: Convert and re-assign the 'runtime' column of the movies dataset into a float. 

In [ ]:
movies['runtime'] = movies['runtime'].astype('float')

Let's now consider the second data type conversion that focuses on time. 
<br>
<br>
Functions such as pd.to_numeric(), pd.to_datetime(), and pd.to_timedelta() are what are called conversion helper functions. They allow you to be more specific about data types with less code. For example:

In [ ]:
print(pd.to_datetime('2019-10-7'))

In [ ]:
pd.to_datetime('20191007',format='%Y%d%m')

Let's convert the 'released' column of our movies dataset to a datetime object from a string. 

In [ ]:
type(movies['released'][0])

In [ ]:
movies['released'] = pd.to_datetime(movies['released'])

In [ ]:
type(movies['released'][0])

In [ ]:
movies['released'][0]

## Another Important Time Function: .dt"


The '.dt' function is useful for getting specific parts of the datetime object. For example, if you call df['column'].dt.weekday_name, it will return a series of days (Sunday, Monday, etc.). Here are some of the other dt attributes:

nanosecond, microsecond, second, minute, hour, day, week, month, quarter, year

For example, if we want to extract the months from the movies['released'] column, we would call: 

In [ ]:
movies['released'].dt.month

## Columns: loc/iloc
<br>
What if we want to exclude high grossing movies from our dataset or hate a particular director?
<br>
We have the pandas dataframe loc/iloc functions!
<br>
Selection Options
There’s 2 main options to achieve selection and indexing activities in Pandas, which can be confusing.
<br>
Selecting data by row numbers (.iloc)
<br>
Selecting data by label or by a conditional statment (.loc)
<br>

#### How to Drop rows in DataFrame by conditions on column values:

In [ ]:
# Get indices of movies for which "gross" is greater than 800
indices = movies.loc[movies['gross'] >= 5000000].index
# Delete these row indexes from dataFrame
movies.drop(indices).head()

### How will we find movies made in the United States?
<br>
Looking at the country column, it looks like, the United States is abbreviated as "USA"
<br>
Using loc and/or iloc, get the names of the movies that are made in the USA.

In [ ]:
movies.loc[movies["country"] == "USA"]

# str Function

The str function is useful for selecting particular rows in our dataset in which a string related function is met. 

<br> 
For example, if we want to convert the elements in a particular string column into all lower case letters, we would run something like this:
<br>


In [ ]:
movies['name'].str.lower().head()

However, we must assign the above code to the column in order to actually see the change within the data frame. 

In [ ]:
movies.head()

Suppose we want to output the elements in which the title of a movie contains the words "I". What would that look like? How about "I" or "You"?

In [ ]:
movies[movies['name'].str.contains(' I ')].head()

In [ ]:
movies[movies['name'].str.contains(' I ') | movies['name'].str.contains(' You ') ].head()

# GroupBy
## agg
How do we find out how many movies were released in 2001?
<br>
df.groupby!
We can supply an aggregation functon of our own choice; we will use "list", but we can also find the average 

In [ ]:
movies.groupby("year").agg(list).sample(10)[["name"]]

In [ ]:
movies.groupby("year").count().sample(10)[["name"]]

# Combining Data Frames

## Merge

Often times we would like to merge two dataframes if they have similar attributes or would like to perform more complex analyses with more attributes present. 

In [ ]:
combined = pd.merge(left = movies, right = netflix, left_on = 'name', right_on = 'title', how = 'inner')
combined
#Notice repetitions

In [ ]:
#Remove repetitive columns
combined = combined.drop(columns=['title', 'rating_x', 'release year'], axis = 1)  #SPECIFY AXIS =1 TO INDICATE COLUMNS
combined

In [ ]:
#rename columns
combined.rename(columns={"rating_y": "rating"})

In [ ]:
#remove repetitive rows 
combined.drop_duplicates(subset=['name'], keep='first')

**Class Question**: What do you think would be outputted if we set 'how' to 'outer'?

Solution: We would get a union of all rows in movies and netflix. The amount of rows in this dataframe would the sum of the rows. We would get many rows with NaNs because some columns in one dataframe aren't present in the other. 

In [ ]:
combined_outer = pd.merge(left = movies, right = netflix, left_on = 'name', right_on = 'title', how = 'outer')
combined_outer

**Class Exercise**: Budget column discussion (What can we do about the 0s in the budget column? Why do you think there are so many zeros?)

Possible Solutions: That info isn't publicly available, so 0 is used as a filler. We could perhaps take the gross column and round to the nearest millionth. We could also average the budget column and fill each 0 with the average. We could get rid of rows with budget 0 but since a majority of rows have budget of 0, it would definitely affect future analysis. 

In [ ]:
len(combined[combined['budget'] == 0])

## Concat

In [ ]:
movies_and_netflix = pd.concat([movies, netflix], axis = 1)
movies_and_netflix.head()
#Notice that column names don't change at all (ratings)
#default of "outer" join, but it is the same length as movies because it takes the indices of just the movies dataset because it was the first dataset passed in

In [ ]:
len(movies_and_netflix) == len(movies)

## Apply
What if we need to modify the dataframe or a column?
<br>
We will use the pandas dataframe apply function!
<br>
df.apply(func, axis=0, broadcast=None, raw=False, reduce=None, result_type=None, args=(), \*\*kwds)
<br>
<br>
**func:** Function to be applied to each column or row. This function accepts a series and returns a series.
<br>
**axis:** Axis along which the function is applied in dataframe. Default value 0.
1. If value is 0 then it applies function to each column.
2. If value is 1 then it applies function to each row.
<br>

**args:** tuple / list of arguments to passed to function.
<br>
For example, we can convert a column to string!

In [ ]:
movies["year"] = movies["year"].apply(func = str)

In [ ]:
type(movies["year"][0])

In [ ]:
movies["year"] = "your code here" # convert the column back to an integer-type column

In [ ]:
type(movies["year"][0])

What other uses for apply can you think of? We can define our own functions, using lambda.

# Indexes in Data Frames

In [ ]:
netflix.set_index('title').head()

In [ ]:
movies.set_index(['country', 'rating'])

In [ ]:
movies.head()

In [ ]:
movies.reset_index()

# Pivot Tables
<br>
A simple example:

In [ ]:
# Create a simple dataframe 
  
# creating a dataframe 
df = pd.DataFrame({'Name': ['John', 'Boby', 'Mina'], 
      'Degree': ['Masters', 'Graduate', 'Graduate'], 
      'Age': [27, 23, 21]}) 
  
df 

In [ ]:
# values can be an object or a list 
df.pivot('Name', 'Degree', 'Age')

In [ ]:
# value is a list 
df.pivot(index ='Name', columns ='Degree', values = ['Age']) 

### Let's try it with the movies dataframe!
<br>
We will use the movies dataframe, because it has a unique index for each row.

In [ ]:
movies.pivot(columns = "company", values = ["name", "year"]).head()

In [ ]:
movies_2 = movies.drop_duplicates(subset = "name", keep='first', inplace=False)
movies_2.pivot("name", "country", "rating")

As you can see, this table is not very informative. How should we proceed?

In [ ]:
movies.pivot_table(index = ["director", "genre", "name"], aggfunc=list).sample(10)

pivot_table is a simplified version of pivot, you supply the index, and aggregation function. The information in the above table is the same as the information in the original dataframe, but it's easier to read.

# Getting started with visualization libraries in python:

#### Seaborn is a popular visualization library commonly used in conjunction with pandas
<br>
We will use it to learn more about our movies dataset

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# simple plot, lets learn about how well the movies in our dataset have done, by genre
plt.figure(figsize=(15, 7))
sns.scatterplot(x = "budget", y = "gross", data = movies.sample(100), hue = "genre")

Because of overplotting, we cannot appropriately compare which genres perform well. We can use other visualizations to learn more about our dataset.

In [ ]:
# Let's try a violin plot, its function is similar to a box and whisker plot, which is to visualize the distribution of a category
plt.figure(figsize=(50, 20))
sns.violinplot(x = "genre", y = "gross", data = movies, hue = "genre")
plt.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)

In [ ]:
plt.figure(figsize=(15, 10))
# learn more about each category
sns.scatterplot(x = "budget", y = "gross", data = movies[movies["genre"] == "Adventure"])

In [ ]:
plt.figure(figsize=(20, 5))
sns.barplot(x = "rating", y = "votes", data = movies) # switch y around to learn more